In [ ]:
# importing libraries
import time
import concurrent.futures
import urllib.request
from bs4 import BeautifulSoup
import sqlite3
import queue
import requests
import json
# setting up connection to database with WAL mode and manual commit
conn = sqlite3.connect('../db.sqlite3')
conn.isolation_level=None
cur = conn.cursor()
cur.execute("PRAGMA journal_mode=WAL")
API_KEY="9578e593&i"
MAX_THREADS = 10
BUFFER_SIZE = 100
get_query = f'''
    select tconst from coreapi_titlebasics where imagelink is null limit {BUFFER_SIZE}
    '''
set_query = "update coreapi_titlebasics set imagelink = ? where tconst = ?"

q = queue.Queue()

In [ ]:
# function which consumes the list and insert values to database in bulk
def insert_imagelinks_todb():
    image_list = []
    print("q size  = ",q.qsize())
    while not q.empty():
        image_list.append(q.get())
    try:
        cur.execute("begin")
        cur.executemany(set_query, image_list)
        cur.execute("commit")
    except:
        cur.execute("rollback")
    image_list.clear()

# task to download and scrape data from imdb
def download_url(imdbid):
    movie_url = f"http://www.omdbapi.com/?apikey={API_KEY}&i="+imdbid[0]
    response = requests.get(movie_url)
    img_link = 'N/A'
    img_link = json.loads(response.text)['Poster']
#     if img_link == 'N/A':
#         img_link = "Not Available"
    q.put((img_link, imdbid[0]))
    
# to download one batch of images
def download_image_links(imdb_ids):
    threads = min(MAX_THREADS, len(imdb_ids))
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_url, imdb_ids)
    insert_imagelinks_todb()

def main():
    while True:
        cur.execute(get_query)
        imdb_ids = cur.fetchall()
        if len(imdb_ids):
            t0 = time.time()
            download_image_links(imdb_ids)
            t1 = time.time()
            print(f"\r {t1-t0} seconds to insert {len(imdb_ids)} links.", end='')
        else:
            break

In [ ]:
main()
conn.close()